# DReS-FL Experiment 

- K = 1
- T = 1
- input dimension = 1 -> scalar
- linear regression
- w = 1
- finite field size = 5
- inputs are selected from {0, 1}
- no relationship between x and y
- the gradient is supposed to be revealed at the end of each training round

Basically, our objective function is $l = (y - xw)^2$ and our gradient is $g = -2(xy - x^2w)$ 

In [10]:
import numpy as np
from lcc.polynomials import LCCPoly, InterpolatedPoly
from mutual_information.estimators.neural.benchmark import neural_estimator_benchmark
from lcc.domain import create_lcc_gradient_domain_basic_setup_no_relationship
from mutual_information.exhaustive_search_mutual_information import calculate_mutual_information_domain

In [11]:
prime = 5
data_range = 2
para_param = 1 # K
priv_param = 1 # T

num_of_samples = 10000
beta_arr = [0, 1]
alpha_arr = [2, 3, 4] # number of clients = 3
weight = 1

In [12]:
def create_encoded_dataset(encoded_data_pol, encoded_label_pol, alpha):
    encoded_dataset = np.empty((len(encoded_data_pol), 2)) 
    for idx, (data_pol, label_pol) in enumerate(zip(encoded_data_pol, encoded_label_pol)):
        encoded_dataset[idx][0] = data_pol(alpha)
        encoded_dataset[idx][1] = label_pol(alpha)
    return encoded_dataset

def calculate_gradient(encoded_dataset, curr_weight, p):
    encoded_data = encoded_dataset[:, 0].reshape(-1, 1)
    encoded_label = encoded_dataset[:, 1].reshape(-1, 1)
    
    gradient = (encoded_label - encoded_data @ curr_weight) % p
    gradient = (encoded_data.T @ gradient) % p
    gradient = (-2 * gradient) % p
    return gradient

def calculate_gradient_samplewise(encoded_dataset, curr_weight, p):
    # g = -2(xy - x^2w)
    gradient = np.empty(encoded_dataset.shape[0])
    for idx, encoded_sample in enumerate(encoded_dataset):
        data, label = encoded_sample[0], encoded_sample[1]
        gradient[idx] = (label - data * curr_weight) % p
        gradient[idx] = (data * gradient[idx]) % p
        gradient[idx] = (-2 * gradient[idx]) % p
    gradient = gradient.astype(int)
    return gradient

In [13]:
secret_data = np.random.randint(low=0, high=2, size=(num_of_samples, 1))
secret_label = np.random.randint(low=0, high=2, size=(num_of_samples, 1))
encoded_secret_data_pol = [LCCPoly(beta_arr, [x[0]], para_param, priv_param, prime) for x in secret_data]
encoded_secret_label_pol = [LCCPoly(beta_arr, [x[0]], para_param, priv_param, prime) for x in secret_label]

In [14]:
# client 0
client_0_encoded_data = create_encoded_dataset(encoded_secret_data_pol, encoded_secret_label_pol, alpha_arr[0])
client_0_encoded_gradient = calculate_gradient_samplewise(client_0_encoded_data, weight, prime)

# client 1
client_1_encoded_data = create_encoded_dataset(encoded_secret_data_pol, encoded_secret_label_pol, alpha_arr[1])
client_1_encoded_gradient = calculate_gradient_samplewise(client_1_encoded_data, weight, prime)

# client 2
client_2_encoded_data = create_encoded_dataset(encoded_secret_data_pol, encoded_secret_label_pol, alpha_arr[2])
client_2_encoded_gradient = calculate_gradient_samplewise(client_2_encoded_data, weight, prime)

In [15]:
revealed_poly_arr = [] 
revealed_gradients = np.empty((client_0_encoded_gradient.shape[0], 1))
revealed_random = np.empty((client_0_encoded_gradient.shape[0], 1))
revealed_poly_constructed_coeff = np.empty((client_0_encoded_gradient.shape[0], 2 * para_param + 1))
for gradient_idx in range(client_0_encoded_gradient.shape[0]):
    revealed_poly = InterpolatedPoly([int(client_0_encoded_gradient[gradient_idx]), int(client_1_encoded_gradient[gradient_idx]), int(client_2_encoded_gradient[gradient_idx])], alpha_arr, prime)
    revealed_gradients[gradient_idx][0] = revealed_poly(beta_arr[0])
    revealed_random[gradient_idx][0] = revealed_poly(beta_arr[1])
    revealed_poly_constructed_coeff[gradient_idx] = revealed_poly.coefficients
    revealed_poly_arr.append(revealed_poly)
    

In [16]:
# create dataset
resulting_dataset = np.concatenate([secret_data, secret_label, revealed_gradients, revealed_random, revealed_poly_constructed_coeff], axis=1)

In [ ]:
estimators_all, results_all = neural_estimator_benchmark(resulting_dataset[:, :2], resulting_dataset[:, 4:])

 12%|██████▋                                              | 1250/10000 [02:47<19:31,  7.47step/s, test=0.89, train=0.91]


Donker Varadhan estimator: 0.89


 10%|█████▏                                              | 1000/10000 [00:05<00:51, 174.47step/s, test=0.87, train=0.90]


MINE estimator: 0.91


 18%|█████████▎                                           | 1750/10000 [03:40<17:21,  7.92step/s, test=0.90, train=0.92]


InfoNCE estimator: 0.90


  0%|▎                                                       | 45/10000 [00:03<11:06, 14.95step/s, test=???, train=0.74]

In [ ]:
estimators_revealed_gradient, results_revealed_gradient = neural_estimator_benchmark(resulting_dataset[:, :2], resulting_dataset[:, 2].reshape(-1, 1))

In [ ]:
basic_experiment_domain = create_lcc_gradient_domain_basic_setup_no_relationship(data_range, beta_arr, alpha_arr, weight, prime)
all_coeff_mi = calculate_mutual_information_domain(basic_experiment_domain, [0, 2], [4, 5, 6, 7, 8], [data_range, data_range], [prime, prime, prime, prime, prime])
revealed_gradient_mi = calculate_mutual_information_domain(basic_experiment_domain, [0, 2], [4], [data_range, data_range], [prime])
print(all_coeff_mi, revealed_gradient_mi)